# **LTI&trade; Advantage** bootcamp notebook for Tool

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [207]:
import requests
import json
import jwt
import base64
from time import time
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown, HTML, Javascript

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [208]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "0",
    "deployment_id": "deployment_0",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "SF5F2Dp1iwNZV_mFa5i0p-wsYbjo1RbLqSCY6c827OVL3G2Qu24dvUXijIs3LsWSkhcR0XzzzAOb0sJ4wxWjXyDOwrV6EQMZGXBXPRhYFJQG-k6js7_FOzsARUnTkNQ43aLkRvjYyQPT3Pn39NJPdnt74SbfPaJ_w4AI9iP0f8Ie-ueKEqnyp2gJvFIl7SgfMJnK2hLPtxOJEqTTR_kAedDKzlxyHg9z96b_7xi1pB3vRi84OfuDvYk66o93dnUV-oflhK3GV4PAyA417gegvR_bo8N2ZtbVMJYtVqVoUPj2fH-L62dHOxz-IyQzC8fdTc01gg-dAqoDQKql4FSAAQ=="
    }
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [209]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "4CNA0Cu38ux_GZNf8ZvFCHxzX4pfzMqnOvls8LbBQg4Gys_E45xuqPeEQSZ3reDXOHpCqEYiz1FKxi5PfQjJVq4DqCkgOWvZo7qKh2LlG8xeautms4IaVqgxHibiNxKRWJ0oYF2flxKBU5XSRVaIXftg2aCplU__tEKbLKxHYJAK23bnvig8Upmo2YWQ_-70P_AGeVCgfNh7ZKWDDJndJk8teyuymj1Csakm8iqJW8Ogv6-hcw19NQGOSpz8xX6UEUx8R1eCmZXSydTvOHmAdukXjKAPr4DaldswpXWJfaXIWa2t2tKzBJF1-_YwLXZXMUYDokklZf0j60ELvBp4VQ==",
            "kid": "1517791263_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "vlI9gOIjCkgAbe687rDGROsuU9cqNfB19OPAr5j-pKL5iaT5HbwEKWfaRaMqnksZzne0_7p5anpjx1dF_KvoQ4dA11403bp0W6QfyrfpE4XMh3FshcoLYtARAmxTVXFNj-fHFPiJileR6UegUA2WIyE5m3Hk5xm3OneWuLcSkBNZ5gKYmhxWgVQLrRe7zYwUoIcMZrbRC0eQ8IyR81A7F6KgR-HSFNBsyROg81R4w3KsJ_YHsGulegVRFz3ow-IUc7gBQ3ce2-xW9CJOqRUoTePzE7nbp9n0KtfaOPwmMSyQVvLkaP3jUchs6A-W2ToG7V-FC4I-XmjqA6oRfNn9CQ==",
            "kid": "1517791263_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "rSht-gF6HDZWE3pYezg7yUdhvPu6viz6xgDhOog3QOGxLOqkJjH2Rrh82xYnKg3i7K1wMVsjKD7J7lnw2RoaGUvovXrMKW95Irlpy_JJ2Xo7VQRb9s_8AMzIOmmjWuVH-6gfLXaaccNU1ITMRtFvStgFIVG5kmADxNmbH_UnRrkT_KJLQIKnbG6k2vAZaI_fzc5TbEhPBCwG0dnRwMD4xAP37wXztkBFPp37QeOMHv9woob6xV6srIVOptlpp3BbTxGhCg2CaDJ2xRItq1UtUDI-KBPTc-71MF1OHBidqt7e7IhNtslYBEnDlFYlb0lWGgernu4Psb3e4eSTuTwAOw==",
            "kid": "1517791263_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "qq0sw3rV0fxjSxQJ3-O06HiXUnEW31Rx6zM4FAXV9dnUnIdnRv6xEDuhcuj4FezE2NPCUh2cQl7Jv4QpwpRPCchV34GbD0gYeE5xz9JwZS3jG08D-waADZxmmnLDTjMe7jmstCxncECRoTuLsfafAJs8h6CpXGxnTritMAbOrntYYHm8NpIac5xDEGtDXnXMN9b_-u2Xd33TVw4L-m_6IkSzs3cb2TIl4gtLsf94QG2ngM5KVq4En3mbJKa-xZHxK5jQUewG1IIRzihi-XWWDpcKkELzj_hU1a3xUkmBhZ42nL4ESUT1LLW2Yu5C-xRXO6dE1A7ZknwrtTToxsV8jw==",
            "kid": "1517791263_3"
        }
    ]
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

<img src="assets/lti_advantage_deeplinking.png" width="60%">

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POST request, so we can build a mock POST request including the parameter `post_data`.

In [210]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

`id_token=<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>
`

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [211]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


DecodeError: Invalid header padding

### Task 2: extract the information needed to render the selector/authoring UI

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [201]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_request')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_media_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
deep_linking_return_url = content_item_message[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(deep_linking_return_url))
mdb('1. Is there any data I must pass back to platform when I return? {0}'.format('data' in deep_linking_claim))
mdb()


1. What is the current course id? efc8b23c-092e-11e8-a22c-c48e8ffb7857
1. What is the current user id? LTIBCU_3
1. Is this user an instructor? True
1. What kind of content item can be created? ['application/vnd.ims.lti.v1.ltilink']
1. Can I return more than one items to be added? True
1. Will the user be prompted before to actually save the items? False
1. Where should I redirect the browser too when done? /tool/efc8b23c-092e-11e8-a22c-c48e8ffb7857/cisr
1. Is there any data I must pass back to platform when I return? True

### Task 3: building the response token

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded (since the request supports multiple content items).

Here we're creating the actual response token.

In [199]:
## First let's create our 2 content items
## Note that the URLs are phony as for now there is now way to launch in the notebook anyway...
simple_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "A simple content item",
      "text": "Some long text",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "key1": "some value"
      }
}

assignment_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "An assignment",
      "text": "Chemical lab sim",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3a",
        "level": "easy"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      }
}

now = int(time())

content_item_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    "http://imsglobal.org/lti/message_type": "DeepLinkingResponse",
    "http://imsglobal.org/lti/version": "1.3.0"
    
    
}

### Task 4: redirect the user back to the platform with the content item selection

Now that we the response token, let's do the actual HTML POST redirection to the platform. Note that because the platform supports `autocreate` there will be no prompt. The 2 items will be added directly to the course.


In [206]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info['client_id'] ,
    'aud': content_item_message['iss']
}

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', 'http://localhost:5000{0}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame');
                $('#deeplinking_frame').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """
autosubmit_js = autosubmit_js.format(deep_linking_return_url)

display(HTML('<iframe id="deeplinking_frame" name="deeplinking_frame" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

<IPython.core.display.Javascript object>